# Distributed Training of Neural Network Models


Below is the basic implementation of Federated Learning code for 2 clients and a server for one communication round using the MNIST dataset.

Your task for this lab is to utilize this code as a benchmark and accomplish  the following objectives:

1. Comment each section of the code to understand its functionality.

2. Experiment with different neural network architectures, varying the number of layers and activation functions

3. Assess the model's performance (accuracy, precision, recall, and f-score) as the number of clients changes. Utilize 2, 4, 6, 8, and 10 clients, and plot a graph illustrating the model's performance across varying client counts.

4. Introduce one dishonest client for a any sepcific number of clients of your choice. For this client, assign some training labels ('y_train') to a specific label, such as '4'. Analyze how the performance ((accuracy, precision, recall, and f-score)) of the global model is affected by the presence of a dishonest client. Find approximately how many labels need to be maliciously altered so that the performance degrades.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import backend as K
import random

In [ ]:
np.random.seed(2324422) # Set seed for numpy. Put your SID instead of number '42'
random.seed(2324422) # Set seed for Python's built-in random number generator.  Put your SID instead of number '42'
tf.random.set_seed(2324422) # Set seed for TensorFlow. Put your SID instead of number '42'
K.clear_session()


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
def create_client_model():
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_client_model(model, x_train, y_train, epochs=5):
    model.fit(x_train, y_train, epochs=epochs, verbose=1)
    return model

2 clients

In [ ]:
x_train_c1, x_train_c2, y_train_c1, y_train_c2 = train_test_split(x_train, y_train, test_size=0.5, random_state=2324422)



In [ ]:
print(x_train_c1.shape)
print(x_train_c2.shape)
print(y_train_c1.shape)
print(y_train_c2.shape)


In [ ]:
# Create and train client models
client1_model = create_client_model()
client2_model = create_client_model()

client1_model = train_client_model(client1_model, x_train_c1, y_train_c1)
client2_model = train_client_model(client2_model, x_train_c2, y_train_c2)

In [ ]:
# Federated averaging
def federated_average_weights(weights1, weights2):
    new_weights = []
    for w1, w2 in zip(weights1, weights2):
        new_weights.append((w1 + w2) / 2)
    return new_weights

In [ ]:
# Aggregate models
aggregated_weights = federated_average_weights(client1_model.get_weights(), client2_model.get_weights())

In [ ]:
# Create global model and set weights
global_model = create_client_model()
global_model.set_weights(aggregated_weights)


In [ ]:
# Evaluate global model
test_loss, test_acc = global_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
prediction = global_model.predict(x_test)
print(prediction.shape)

prediction = list(prediction.argmax(axis=1))

In [ ]:
print(classification_report(y_test, prediction))

In [ ]:
print("Accuracy test set:", accuracy_score(y_test, prediction))
print("precison test set:", precision_score(y_test, prediction, average = 'weighted'))
print("Recall test set:", recall_score(y_test, prediction, average = 'weighted'))
print("f-score test set:", f1_score(y_test, prediction, average = 'weighted'))


In [ ]:
test_precision= []
test_recall = []
test_f1_score = []
test_accuracy=[]

In [ ]:
test_accuracy.append(0.8508)
test_precision.append(0.8820127173009187)
test_recall.append(0.8508)
test_f1_score.append(0.8522835934707446)

In [ ]:
print(test_accuracy)
print(test_precision)
print(test_recall)
print(test_f1_score)

4 clients

In [ ]:
# split data into 4 clients

In [ ]:
client_1 = (x_train[0:15000], y_train[0:15000])
client_2 = (x_train[15000:30000], y_train[15000:30000])
client_3 = (x_train[30000:45000], y_train[30000:45000])
client_4 = (x_train[45000:], y_train[45000:])


In [ ]:
print(client_1[0].shape)
print(client_2[0].shape)
print(client_3[0].shape)
print(client_4[0].shape)

print(client_1[1].shape)
print(client_2[1].shape)
print(client_3[1].shape)
print(client_4[1].shape)

In [ ]:
client1_model = create_client_model()
client2_model = create_client_model()
client3_model = create_client_model()
client4_model = create_client_model()

client1_model = train_client_model(client1_model, client_1[0], client_1[1])
client2_model = train_client_model(client2_model, client_2[0], client_1[1])
client3_model = train_client_model(client3_model, client_3[0], client_1[1])
client4_model = train_client_model(client4_model, client_4[0], client_1[1])

In [ ]:
# Federated averaging
def federated_average_weights(weights1, weights2, weights3, weights4):
    new_weights = []
    for w1, w2, w3, w4 in zip(weights1, weights2,  weights3,  weights4):
        new_weights.append((w1 + w2 + w3 + w4) / 4)
    return new_weights

In [ ]:
# Aggregate models
aggregated_weights = federated_average_weights(client1_model.get_weights(), client2_model.get_weights(), client3_model.get_weights(), client4_model.get_weights())

In [ ]:
# Create global model and set weights
global_model = create_client_model()
global_model.set_weights(aggregated_weights)

# Evaluate global model
test_loss, test_acc = global_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

In [ ]:
prediction = global_model.predict(x_test)
print(prediction.shape)

prediction = list(prediction.argmax(axis=1))

In [ ]:
print("Accuracy test set:", accuracy_score(y_test, prediction))
print("precison test set:", precision_score(y_test, prediction, average = 'weighted'))
print("Recall test set:", recall_score(y_test, prediction, average = 'weighted'))
print("f-score test set:", f1_score(y_test, prediction, average = 'weighted'))

In [ ]:
test_accuracy.append(0.3374)
test_precision.append(0.35501437213213655)
test_recall.append(0.3374)
test_f1_score.append(0.2962132458250437)

In [ ]:
print(test_accuracy)
print(test_precision)
print(test_recall)
print(test_f1_score)

6 clients cell


In [ ]:
client_1 = (x_train[0:10000], y_train[0:10000])
client_2 = (x_train[10000:20000], y_train[10000:20000])
client_3 = (x_train[20000:30000], y_train[20000:30000])
client_4 = (x_train[30000:40000], y_train[30000:40000])
client_5 = (x_train[40000:50000], y_train[40000:50000])
client_6 = (x_train[50000:], y_train[50000:])

In [ ]:
print(client_1[0].shape)
print(client_2[0].shape)
print(client_3[0].shape)
print(client_4[0].shape)
print(client_5[0].shape)
print(client_6[0].shape)

print(client_1[1].shape)
print(client_2[1].shape)
print(client_3[1].shape)
print(client_4[1].shape)
print(client_5[1].shape)
print(client_6[1].shape)

In [ ]:
client1_model = create_client_model()
client2_model = create_client_model()
client3_model = create_client_model()
client4_model = create_client_model()
client5_model = create_client_model()
client6_model = create_client_model()

client1_model = train_client_model(client1_model, client_1[0], client_1[1])
client2_model = train_client_model(client2_model, client_2[0], client_1[1])
client3_model = train_client_model(client3_model, client_3[0], client_1[1])
client4_model = train_client_model(client4_model, client_4[0], client_1[1])
client5_model = train_client_model(client5_model, client_5[0], client_1[1])
client6_model = train_client_model(client6_model, client_6[0], client_1[1])

In [ ]:
# Federated averaging
def federated_average_weights(weights1, weights2, weights3, weights4, weights5, weights6):
    new_weights = []
    for w1, w2, w3, w4, w5, w6 in zip(weights1, weights2,  weights3,  weights4, weights5, weights6):
        new_weights.append((w1 + w2 + w3 + w4 + w5 + w6) / 6)
    return new_weights

In [ ]:
# Aggregate models
aggregated_weights = federated_average_weights(client1_model.get_weights(), client2_model.get_weights(), client3_model.get_weights(), client4_model.get_weights(),client5_model.get_weights(),client6_model.get_weights())

In [ ]:
# Create global model and set weights
global_model = create_client_model()
global_model.set_weights(aggregated_weights)

# Evaluate global model
test_loss, test_acc = global_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

In [ ]:
prediction = global_model.predict(x_test)
print(prediction.shape)

prediction = list(prediction.argmax(axis=1))

In [ ]:
print("Accuracy test set:", accuracy_score(y_test, prediction))
print("precison test set:", precision_score(y_test, prediction, average = 'weighted'))
print("Recall test set:", recall_score(y_test, prediction, average = 'weighted'))
print("f-score test set:", f1_score(y_test, prediction, average = 'weighted'))

In [ ]:
test_accuracy.append(0.2073)
test_precision.append(0.2192389494021009)
test_recall.append(0.2073)
test_f1_score.append(0.15575406556726198)

In [ ]:
print(test_accuracy)
print(test_precision)
print(test_recall)
print(test_f1_score)

8 clients


In [ ]:
client_1 = (x_train[0:7500], y_train[0:7500])
client_2 = (x_train[7500:15000], y_train[7500:15000])
client_3 = (x_train[15000:22500], y_train[15000:22500])
client_4 = (x_train[22500:30000], y_train[22500:30000])
client_5 = (x_train[30000:37500], y_train[30000:37500])
client_6 = (x_train[37500:45000], y_train[37500:45000])
client_7 = (x_train[45000:52500], y_train[45000:52500])
client_8 = (x_train[52500:], y_train[52500:]) 

In [ ]:
print(client_1[0].shape)
print(client_2[0].shape)
print(client_3[0].shape)
print(client_4[0].shape)
print(client_5[0].shape)
print(client_6[0].shape)
print(client_7[0].shape)
print(client_8[0].shape)

print(client_1[1].shape)
print(client_2[1].shape)
print(client_3[1].shape)
print(client_4[1].shape)
print(client_5[1].shape)
print(client_6[1].shape)
print(client_7[1].shape)
print(client_8[1].shape)

In [ ]:
client1_model = create_client_model()
client2_model = create_client_model()
client3_model = create_client_model()
client4_model = create_client_model()
client5_model = create_client_model()
client6_model = create_client_model()
client7_model = create_client_model()
client8_model = create_client_model()

client1_model = train_client_model(client1_model, client_1[0], client_1[1])
client2_model = train_client_model(client2_model, client_2[0], client_2[1])
client3_model = train_client_model(client3_model, client_3[0], client_3[1])
client4_model = train_client_model(client4_model, client_4[0], client_4[1])
client5_model = train_client_model(client5_model, client_5[0], client_5[1])
client6_model = train_client_model(client6_model, client_6[0], client_6[1])
client7_model = train_client_model(client7_model, client_7[0], client_7[1])
client8_model = train_client_model(client8_model, client_8[0], client_8[1])

In [ ]:
# Federated averaging
def federated_average_weights(weights1, weights2, weights3, weights4,  weights5,  weights6,  weights7,  weights8):
    new_weights = []
    for w1, w2, w3, w4, w5, w6, w7, w8 in zip(weights1, weights2,  weights3,  weights4,  weights5,  weights6,  weights7,  weights8):
        new_weights.append((w1 + w2 + w3 + w4 + w5 + w6 + w7 + w8) / 8)
    return new_weights

In [ ]:
# Aggregate models
aggregated_weights = federated_average_weights(client1_model.get_weights(), client2_model.get_weights(), 
                                               client3_model.get_weights(), client4_model.get_weights(),  
                                               client5_model.get_weights(), client6_model.get_weights(), 
                                               client7_model.get_weights(), client8_model.get_weights())

In [ ]:
# Create global model and set weights
global_model = create_client_model()
global_model.set_weights(aggregated_weights)

# Evaluate global model
test_loss, test_acc = global_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

In [ ]:
prediction = global_model.predict(x_test)
print(prediction.shape)

prediction = list(prediction.argmax(axis=1))

In [ ]:
print("Accuracy test set:", accuracy_score(y_test, prediction))
print("precison test set:", precision_score(y_test, prediction, average = 'weighted'))
print("Recall test set:", recall_score(y_test, prediction, average = 'weighted'))
print("f-score test set:", f1_score(y_test, prediction, average = 'weighted'))

In [ ]:
test_accuracy.append(0.4496)
test_precision.append(0.7342762826710872)
test_recall.append(0.4496)
test_f1_score.append(0.41847081461118935)

In [ ]:
print(test_accuracy)
print(test_precision)
print(test_recall)
print(test_f1_score)

10 clients

In [ ]:
client_1 = (x_train[0:6000],y_train[0:6000])
client_2= (x_train[6000:12000],y_train[6000:12000])
client_3= (x_train[12000:18000],y_train[12000:18000])
client_4= (x_train[18000:24000], y_train[18000:24000])
client_5=(x_train[24000:30000], y_train[24000:30000])
client_6= (x_train[30000:36000], y_train[30000:36000])
client_7=(x_train[36000:42000], y_train[36000:42000])
client_8=(x_train[42000:48000], y_train[42000:48000])
client_9=(x_train[48000:54000], y_train[48000:54000])
client_10=(x_train[54000:],y_train[54000:])

In [ ]:
print(client_1[0].shape)
print(client_2[0].shape)
print(client_3[0].shape)
print(client_4[0].shape)
print(client_5[0].shape)
print(client_6[0].shape)
print(client_7[0].shape)
print(client_8[0].shape)
print(client_9[0].shape)
print(client_10[0].shape)

print(client_1[1].shape)
print(client_2[1].shape)
print(client_3[1].shape)
print(client_4[1].shape)
print(client_5[1].shape)
print(client_6[1].shape)
print(client_7[1].shape)
print(client_8[1].shape)
print(client_9[1].shape)
print(client_10[1].shape)

In [ ]:
client1_model = create_client_model()
client2_model = create_client_model()
client3_model = create_client_model()
client4_model = create_client_model()
client5_model = create_client_model()
client6_model = create_client_model()
client7_model = create_client_model()
client8_model = create_client_model()
client9_model = create_client_model()
client10_model = create_client_model()

client1_model = train_client_model(client1_model, client_1[0], client_1[1])
client2_model = train_client_model(client2_model, client_2[0], client_2[1])
client3_model = train_client_model(client3_model, client_3[0], client_3[1])
client4_model = train_client_model(client4_model, client_4[0], client_4[1])
client5_model = train_client_model(client5_model, client_5[0], client_5[1])
client6_model = train_client_model(client6_model, client_6[0], client_6[1])
client7_model = train_client_model(client7_model, client_7[0], client_7[1])
client8_model = train_client_model(client8_model, client_8[0], client_8[1])
client9_model = train_client_model(client8_model, client_9[0], client_9[1])
client10_model = train_client_model(client8_model, client_10[0], client_10[1])

In [ ]:
# Federated averaging
def federated_average_weights(weights1, weights2, weights3, weights4,  weights5,  weights6,  weights7,  weights8, weights9, weights10):
    new_weights = []
    for w1, w2, w3, w4, w5, w6, w7, w8, w9, w10 in zip(weights1, weights2,  weights3,  weights4,  weights5,  weights6,  weights7,  weights8, weights9, weights10):
        new_weights.append((w1 + w2 + w3 + w4 + w5 + w6 + w7 + w8 + w9 + w10) / 10)
    return new_weights

In [ ]:
# Aggregate models
aggregated_weights = federated_average_weights(client1_model.get_weights(), client2_model.get_weights(), 
                                               client3_model.get_weights(), client4_model.get_weights(),  
                                               client5_model.get_weights(), client6_model.get_weights(), 
                                               client7_model.get_weights(), client8_model.get_weights(),
                                               client9_model.get_weights(), client10_model.get_weights())
# Create global model and set weights
global_model = create_client_model()
global_model.set_weights(aggregated_weights)


In [ ]:
# Evaluate global model
test_loss, test_acc = global_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

prediction = global_model.predict(x_test)
print(prediction.shape)

prediction = list(prediction.argmax(axis=1))

In [ ]:
print("Accuracy test set:", accuracy_score(y_test, prediction))
print("precison test set:", precision_score(y_test, prediction, average = 'weighted'))
print("Recall test set:", recall_score(y_test, prediction, average = 'weighted'))
print("f-score test set:", f1_score(y_test, prediction, average = 'weighted'))

print(classification_report(y_test, prediction))

In [ ]:
test_accuracy.append(0.6243)
test_precision.append(0.8546927376488528)
test_recall.append(0.6243)
test_f1_score.append(0.6157209884017654)

print(test_accuracy)
print(test_precision)
print(test_recall)
print(test_f1_score)

Plot the model's performance for the number of clients ranging from 2, 4, 6, 8, and 10.¶

In [ ]:
import pandas as pd

In [ ]:
ErrorMetrics = []
ErrorMetrics.append(test_accuracy)
ErrorMetrics.append(test_precision)
ErrorMetrics.append(test_recall)
ErrorMetrics.append(test_f1_score)

print(ErrorMetrics)

ErrorMetrics_pd = pd.DataFrame(ErrorMetrics)
ErrorMetrics_pd.columns = [2,4,6,8,10]
ErrorMetrics_pd = ErrorMetrics_pd.T
ErrorMetrics_pd.columns = ['Accuracy','Precision','Recall','F1_Score']
ErrorMetrics_pd

In [ ]:
ErrorMetrics_pd.plot.bar(width=0.7, alpha=0.7)

Dishonest 


In [ ]:
# Split data into 4 clients
client_1 = (x_train[0:15000], y_train[0:15000])
client_2 = (x_train[15000:30000], y_train[15000:30000])
client_3 = (x_train[30000:45000], y_train[30000:45000])
client_4 = (x_train[45000:], y_train[45000:])

In [ ]:
client_3[1].size

In [ ]:
client_3[1]

In [ ]:
client_3[1][:3000] = 5

In [ ]:
client_3[1]

In [ ]:
print(client_1[0].shape)
print(client_2[0].shape)
print(client_3[0].shape)
print(client_4[0].shape)

print(client_1[1].shape)
print(client_2[1].shape)
print(client_3[1].shape)
print(client_4[1].shape)

In [ ]:
# Create and train client models
client1_model = create_client_model()
client2_model = create_client_model()
client3_model = create_client_model()
client4_model = create_client_model()

client1_model = train_client_model(client1_model, client_1[0], client_1[1])
client2_model = train_client_model(client2_model, client_2[0], client_1[1])
client3_model = train_client_model(client3_model, client_3[0], client_1[1])
client4_model = train_client_model(client4_model, client_4[0], client_1[1])

In [ ]:
# Federated averaging
def federated_average_weights(weights1, weights2, weights3, weights4):
    new_weights = []
    for w1, w2, w3, w4 in zip(weights1, weights2, weights3, weights4):
        new_weights.append((w1 + w2 + w3 + w4 ) / 4)
    return new_weights


# Aggregate models
aggregated_weights = federated_average_weights(client1_model.get_weights(), 
                                               client2_model.get_weights(), 
                                               client3_model.get_weights(),
                                               client4_model.get_weights())
                                             
# Create global model and set weights
global_model = create_client_model()
global_model.set_weights(aggregated_weights)

In [ ]:
# Evaluate global model
test_loss, test_acc = global_model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

predictions = global_model.predict(x_test)
print(predictions.shape)

predictions = list(predictions.argmax(axis=1))

print("Accuracy test set: ", accuracy_score(y_test, predictions))

print("Precision test set: ", precision_score(y_test, predictions, average = 'weighted' )) 
print("Recall test set: ", recall_score(y_test, predictions, average = 'weighted' ))
print("F-score test set: ", f1_score(y_test, predictions, average = 'weighted' ))

print(classification_report(y_test, predictions))

In [ ]:
test_accuracy_dishonest  = []
test_precision_dishonest = []
test_recall_dishonest    = []
test_f1_score_dishonest  = []

In [ ]:
test_accuracy_dishonest.append(test_accuracy[2])
test_precision_dishonest.append(test_precision[2])
test_recall_dishonest.append(test_recall[2])
test_f1_score_dishonest.append(test_f1_score[2])

test_accuracy_dishonest.append(0.3603)
test_precision_dishonest.append(0.423169487718241)
test_recall_dishonest.append(0.3603)
test_f1_score_dishonest.append(0.3296364992938911)

print(test_accuracy_dishonest)
print(test_precision_dishonest)
print(test_recall_dishonest)
print(test_f1_score_dishonest)

In [ ]:
### Plot the model's performance for the 4 clients, one of them (number=3) is dishonest

ErrorMetrics = []
ErrorMetrics.append(test_accuracy_dishonest)
ErrorMetrics.append(test_precision_dishonest)
ErrorMetrics.append(test_recall_dishonest)
ErrorMetrics.append(test_f1_score_dishonest)

print(ErrorMetrics)

ErrorMetrics_pd = pd.DataFrame(ErrorMetrics)
ErrorMetrics_pd.columns = ['All are honest', '#3 is dishonest']
ErrorMetrics_pd = ErrorMetrics_pd.T
ErrorMetrics_pd.columns = ['Accuracy','Precision','Recall','F1_Score']
ErrorMetrics_pd

In [ ]:
ErrorMetrics_pd.plot.bar(width=0.7, alpha=0.7)

# Lab Logbook requirement: Document the following in lablogbook

1. Plot the model's performance for the number of clients ranging from 2, 4, 6, 8, and 10.

2. For any specified number of clients, compare the model's performance when one (or more) dishonest clients are present. Plot the performance metrics before and after the presence of dishonest clients.




# Challenges (Optional):

## The above code is for one communication round. Write code for multiple communication rounds. In each communication round, clients will train the model on a new dataset. Then the server will merge all these models. Subsequently, the clients will retrain the merged model in the next communication round.

